In [ ]:
TE = joinpath(tempdir(), "OnePiece.test")

if isdir(TE)

    rm(TE, recursive = true)

end

mkdir(TE)

In [ ]:
#using Revise
using OnePiece

---

In [ ]:
pa = dirname(@__DIR__)

pap = joinpath(pa, "Project.toml")

pas = joinpath(pa, "src")

pat = joinpath(pa, "test")

In [ ]:
sr_ = [na for na in readdir(pas) if isdir(joinpath(pas, na))]

te_ = [splitext(na)[1] for na in readdir(pat) if endswith(na, ".ipynb")]

te_ = [te for te in te_ if !(te in ("runtests",))]

symdiff(sr_, te_)

In [ ]:
for (id, te) in enumerate(te_)

    println(id)

    nb = joinpath(pat, "$te.ipynb")

    if id < 1

        println("Skipped.")

        continue

    end

    println("Running ", nb, " (", id, ")")

    run(
        `jupyter-nbconvert --execute --ExecutePreprocessor.timeout=-1 --clear-output --inplace $nb`,
    )

end

---